In [1]:
import os
import pandas as pd
from glob import glob

# Directory containing .parquet files
parquet_dir = 'EGKK'  # Replace with actual path

# Get all .parquet files in the directory
parquet_files = glob(os.path.join(parquet_dir, '*.parquet'))

# Initialize a dictionary to store flight counts for each file
file_flight_counts = {}

# Function to collect unique icao24 values
def collect_unique_icao24(parquet_files):
    unique_icao24 = set()
    for file in parquet_files:
        df = pd.read_parquet(file)
        unique_icao24.update(df['icao24'].unique())
    return list(unique_icao24)

# Process each .parquet file
for file in parquet_files:
    # Read the parquet file
    df = pd.read_parquet(file)
    
    # Count the number of unique flights in this file
    # Use either icao24 or callsign, whichever is more appropriate
    flight_count = len(df['icao24'].unique())  # or use df['callsign'].unique()
    
    # Store the count in the dictionary
    file_flight_counts[os.path.basename(file)] = flight_count

columns = df.columns
print(columns)
# Convert the dictionary to a DataFrame for easier viewing
result_df = pd.DataFrame.from_dict(file_flight_counts, orient='index', columns=['flight_count'])
result_df.index.name = 'file_name'
result_df = result_df.sort_values('flight_count', ascending=False)

print(f"Total number of files processed: {len(result_df)}")
print("\nTop 10 files by flight count:")
print(result_df.head(10))

# Total number of flights
total_flights = result_df['flight_count'].sum()
print(f"\nTotal number of flights across all files: {total_flights}")

# Collect unique icao24 values
unique_icao24_list = collect_unique_icao24(parquet_files)
print(f"\nTotal number of unique icao24 values: {len(unique_icao24_list)}")


Index(['time', 'icao24', 'lat', 'lon', 'velocity', 'heading', 'vertrate',
       'callsign', 'onground', 'alert', 'spi', 'squawk', 'baroaltitude',
       'geoaltitude', 'lastposupdate', 'lastcontact', 'serials', 'hour'],
      dtype='object')
Total number of files processed: 31

Top 10 files by flight count:
                                flight_count
file_name                                   
flight_data_2024-08-25.parquet           297
flight_data_2024-08-04.parquet           296
flight_data_2024-08-11.parquet           294
flight_data_2024-08-18.parquet           292
flight_data_2024-08-09.parquet           292
flight_data_2024-08-29.parquet           292
flight_data_2024-08-28.parquet           290
flight_data_2024-08-08.parquet           289
flight_data_2024-08-14.parquet           289
flight_data_2024-08-19.parquet           288

Total number of flights across all files: 8814

Total number of unique icao24 values: 1505


In [2]:
import pandas as pd
import numpy as np
def warn_and_fill(bad_line):
    # print(f"Filling line with NaN: {bad_line}")
    return pd.Series([np.nan] * len(bad_line))

df = pd.read_csv('aircraftDatabase.csv', 
                 on_bad_lines=warn_and_fill, 
                 engine='python')
df.columns = df.columns.str.strip("'")
print(df.columns)
# Filter the DataFrame to include only the aircraft in unique_icao24_list
filtered_df = df[df['icao24'].isin(unique_icao24_list)]

# Count the occurrences of each aircraft type for typecode, icaoaircrafttype, and categoryDescription
aircraft_type_counts = {
    'typecode': filtered_df['typecode'].value_counts(),
    'icaoaircrafttype': filtered_df['icaoaircrafttype'].value_counts(),
    'categoryDescription': filtered_df['categoryDescription'].value_counts(),
    'model':filtered_df['model'].value_counts()
}

for column, counts in aircraft_type_counts.items():
    print(f"\n{column} Distribution:")
    print(counts)

    # Calculate the percentage of each aircraft type
    percentages = (counts / len(filtered_df)) * 100
    print(f"\n{column} Distribution (Percentage):")
    print(percentages)

# Check for any icao24 values that are in unique_icao24_list but not in df
missing_icao24 = set(unique_icao24_list) - set(df['icao24'])

if missing_icao24:
    print(f"\nNumber of icao24 values not found in the aircraft database: {len(missing_icao24)}")
    print("First few missing icao24 values:", list(missing_icao24)[:5])
else:
    print("\nAll icao24 values from the flight data were found in the aircraft database.")


def identify_wtc(typecode):
    # Define WTC categories
    jumbo = ['A388']
    heavy = ['A359', 'B788', 'B789', 'B77W', 'B772', 'B763', 'A332', 'A333', 'A339', 'B77L', 'A343', 'A310', 'B752', 'B773', 'B78X']
    medium = ['A320', 'B738', 'A319', 'A21N', 'A20N', 'B38M', 'A321', 'BCS3', 'E190', 'B739', 'E295', 'DH8D', 'E195', 'AT76', 'B39M', 'AT75', 'B737', 'CRJ2', 'E290', 'E145', 'SB91']
    light = ['B06', 'C42', 'DA42', 'C208', 'CL60', 'PC12', 'E35L', 'C68A', 'PA46', 'SPIT', 'CRUZ', 'C152', 'P68', 'GLF6', 'EC35', 'C25M', 'PA34', 'C172']
    
    # Convert typecode to uppercase for case-insensitive comparison
    # typecode = typecode.upper()
    
    if typecode in jumbo:
        return 'J'
    elif typecode in heavy:
        return 'H'
    elif typecode in medium:
        return 'M'
    elif typecode in light:
        return 'L'
    else:
        return 'Unknown'

# Check the traffic mix from typecode
traffic_mix = filtered_df['typecode'].apply(identify_wtc).value_counts()
print("\nTraffic Mix Distribution:")
print(traffic_mix)

# Calculate the percentage of each aircraft category
traffic_mix_percentage = (traffic_mix / len(filtered_df)) * 100
print("\nTraffic Mix Distribution (Percentage):")
print(traffic_mix_percentage)



Index(['icao24', 'registration', 'manufacturericao', 'manufacturername',
       'model', 'typecode', 'serialnumber', 'linenumber', 'icaoaircrafttype',
       'operator', 'operatorcallsign', 'operatoricao', 'operatoriata', 'owner',
       'testreg', 'registered', 'reguntil', 'status', 'built',
       'firstflightdate', 'seatconfiguration', 'engines', 'modes', 'adsb',
       'acars', 'notes', 'categoryDescription'],
      dtype='object')

typecode Distribution:
typecode
A320         311
B738         263
A319          90
A21N          79
A20N          74
B38M          57
A321          56
B788          56
A359          46
BCS3          29
A388          27
B789          25
B77W          20
E190          18
B772          18
B763          15
AT76          13
A332           9
B39M           7
B739           5
E295           4
DH8D           4
E195           4
A333           3
AT75           2
A339           2
B06            2
B77L           2
E290           2
B737           2
CRJ2           2


In [3]:
print(missing_icao24)

{'4d24b1', '4bb201', '4d2528', '601828', '02019c', '4d24af', '02a261', '4d2490', 'c06a88', '4aca8b', '4bce43', '4bce1a', '4d201d', '4cc57a', '4aca85', '407fc7', '503e99', '4d252a', '4cae63', '4075c2', '781f2e', '4cae92', '502d7a', '4cadef', '4cadea', '471db7', '440117', '4d252b', '02a262', '781f2f', '4bb205', '4bb2b0', '4cadbf', '4d2536', '4d24d8', '4d245b', '440cf8', '4aca90', '76cd48', '503cc6', '4d24d4', '4d242e', '4bb202', '4d2516', '407e9f', '4cade6', '4d2492', '4080f6', '49526d', '4cc582', '34744d', '4caddf', 'a1d99b', '4aca8c', '4cadaa', '4d2527', '408026', '440072', '4d24d5', '4cadfe', '4aca81', '4cc574', 'a4c98f', '4aca88', '4b1a2d', '4010db', '4d200c', '4bb206', '503e9b', '4d248e', '4aca8f', '4d2035', '4b19fe', '503d73', '469e42', '4cae67', '440d6f', '347588', '502d7d', '4b19ef', '40807a', 'a4ccba', '40814a', 'c06aab', '471db5', '502d5f', '407f62', '4d2493', '4cade9', '4bcdaa', '4acb23', 'c03f37', '4cadcb', '471f04', '471f05', '407f67', '4bb209', '407fea', '4d24de', '781fb4',

In [4]:
# Filter out the missing icao24 from the filtered_df
filtered_df_with_icao24 = filtered_df[~filtered_df['icao24'].isin(missing_icao24)]

# Count the number of flights after filtering
num_flights_with_icao24 = len(filtered_df_with_icao24)

print(f"\nNumber of flights after filtering out missing icao24: {num_flights_with_icao24}")
print(f"Number of flights removed: {len(filtered_df) - num_flights_with_icao24}")

# Update the traffic mix calculation with the new filtered dataframe
traffic_mix_updated = filtered_df_with_icao24['typecode'].apply(identify_wtc).value_counts()
print("\nUpdated Traffic Mix Distribution:")
print(traffic_mix_updated)

# Calculate the updated percentage of each aircraft category
traffic_mix_percentage_updated = (traffic_mix_updated / len(filtered_df_with_icao24)) * 100
print("\nUpdated Traffic Mix Distribution (Percentage):")
print(traffic_mix_percentage_updated)



Number of flights after filtering out missing icao24: 1278
Number of flights removed: 0

Updated Traffic Mix Distribution:
typecode
M          1024
H           201
J            27
L            19
Unknown       7
Name: count, dtype: int64

Updated Traffic Mix Distribution (Percentage):
typecode
M          80.125196
H          15.727700
J           2.112676
L           1.486698
Unknown     0.547731
Name: count, dtype: float64


In [5]:
# from ATMAP_Europe import metar_to_csv

input_file = 'egkk.txt'
output_file = 'EGKK_2024_score.csv'
# metar_to_csv(input_file, output_file)


In [6]:
# from ATMAP_Europe import visualize_metar_scores
csv_file = 'EGKK_2024_score.csv'
output_image = 'metar_visualization.png'
# visualize_metar_scores(csv_file, output_image)